In [1]:
# from analytics.bamboo import Bamboo as bb
#import pvc
#from bento import wait

import os
import pandas as pd
import numpy as np
from statistics import median
#from datetime import date, timedelta
from  nltk.metrics import agreement

#import re
#from timeit import default_timer as timer

# import altair as alt
# alt.data_transformers.disable_max_rows()

import matplotlib.pyplot as plt
import seaborn as sns
#sns.set_context('poster')
#mpl.style.use('seaborn-whitegrid')
sns.set(context='poster', font_scale=0.5, style='whitegrid', palette="colorblind")

# # pd.set_option('display.max_colwidth', -1)
# pd.options.display.max_rows = 100
# pd.options.display.max_columns = 10000
# # pd.set_option('display.width', 1000)
# pd.options.mode.chained_assignment = None  # default='warn'

# from IPython.core.display import display, HTML, clear_output   # %config InlineBackend.figure_format = 'retina'
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [93]:
pwd

'/private/home/dlicht/fairseq-py/examples/nllb/evaluation/XSTS_eval/2021_09_15_next_14/analysis_notebooks'

In [95]:
write_file = False
output_folder   = ''
output_filename = 'merged_data_2021_09_13.tsv' # for upload at the end

In [3]:
# def Load_Data(sql,namespace='aml'):
#     """Pull the data from HIVE using Bambo, the query may take several minutes"""
#     from timeit import default_timer as timer
#     start = timer()

#     df_RAW = bb.presto(query = sql, namespace = namespace)

#     end = timer()
#     print(f"Presto Query and data pull took {round(end-start,1)} seconds")
#     return df_RAW

# Load Real Annotation Data
from "Offline" tsv files

## First, loading the ID mappings 

In [4]:
# sql = f"""

# select 
#     CAST (fbid AS BIGINT) AS fbid,
#     id,
#     sentence1_lang,
#     sentence2_lang
#     -- sentence1,
#     -- sentence2
# from test_NLLB_id_mapping_1a

# """

# df_id_map = Load_Data(sql, namespace='aml')
# df_id_map.info()

In [5]:
DIR = "/private/home/dlicht/fairseq-py/examples/nllb/evaluation/XSTS_eval/2021_09_15_next_14/human_eval_sample_generation/master_copy"
df_source= pd.DataFrame()

for f in os.listdir(f"{DIR}"):
    print(f)
    try:
        temp_df = pd.read_csv(DIR+"/"+f, sep='\t', error_bad_lines=False)
        # temp_df = pd.read_csv(DIR+"/"+f, sep='\t', on_bad_lines='error', verbose=True)
        print(temp_df.shape)
        temp_df['annotator_lang'] = f[-7:-4]
        df_source = df_source.append(temp_df)
    except:
        print(f"file import ERROR {f}")

df_source.tail()

20210913_eval_NLLB_m2m100_zul.tsv
(4454, 9)
20210913_eval_NLLB_m2m100_urd.tsv
(4454, 9)
20210913_eval_NLLB_m2m100_swh.tsv
(4454, 9)
20210913_eval_NLLB_m2m100_snd.tsv
(4454, 9)
20210913_eval_NLLB_m2m100_slv.tsv
(4454, 9)
20210913_eval_NLLB_m2m100_ron.tsv
(4454, 9)
20210913_eval_NLLB_m2m100_por.tsv
(4454, 9)
20210913_eval_NLLB_m2m100_kat.tsv
(4454, 9)
20210913_eval_NLLB_m2m100_hin.tsv
(4454, 9)
20210913_eval_NLLB_m2m100_bul.tsv
(4454, 9)
20210913_eval_NLLB_m2m100_bos.tsv
(4454, 9)
20210913_eval_NLLB_m2m100_azj.tsv
(4454, 9)
20210913_eval_NLLB_m2m100_ara.tsv
(4454, 9)
20210913_eval_NLLB_m2m100_amh.tsv
(4454, 9)


,fbid,FLORES_ID,sentence1_lang,sentence2_lang,sentence1,sentence2,model,direction,dataset,annotator_lang
4449,15948662869656859630,171,eng,amh,"In late 2015, TogiNet established AstroNet Rad...",በ 2015 መገባደጃ ላይ ቶጊኔት አስትሮኔት ሬዲዮን እንደ አንድ ንዑስ ጣ...,human,forward,devtest,amh
4450,1344353142545256770,611,amh,eng,በአደጉ ሀገራት አንዳንድ ጊዜ ስለ ውሃ ጥራት እና ስለ ድልድዮች መውደቅ ...,In developed countries you seldom hear similar...,human,backward,devtest,amh
4451,12649507862239568202,805,eng,amh,Visibility may also be restricted by falling o...,ዕይታ በሚወድቅ ወይም በሚነፍስ በረዶ ወይም በውሃ መጤዝ ወይም በመኪና መ...,human,forward,devtest,amh
4452,16384303385687987122,647,eng,amh,The fission bomb works on the principle that i...,የፍንዳታ ቦንብ የሚሠራው ሁሉንም ኃይል በሚቻለው ፕሮቶን እና ኒውትሮን ወ...,human,forward,devtest,amh
4453,10869730886167153991,180,eng,amh,The final match of the series will take place ...,የስፕሪንግቦክስ አውስትራሊያ በሚገጥምበት ጊዜ፣ የተከታታይ የመጨረሻ ግጥሚ...,human,forward,devtest,amh


In [6]:
df_source.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62356 entries, 0 to 4453
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   fbid            62356 non-null  uint64
 1   FLORES_ID       62356 non-null  int64 
 2   sentence1_lang  62356 non-null  object
 3   sentence2_lang  62356 non-null  object
 4   sentence1       62356 non-null  object
 5   sentence2       62356 non-null  object
 6   model           62356 non-null  object
 7   direction       62356 non-null  object
 8   dataset         62356 non-null  object
 9   annotator_lang  62356 non-null  object
dtypes: int64(1), object(8), uint64(1)
memory usage: 5.2+ MB


In [7]:
df_source

,fbid,FLORES_ID,sentence1_lang,sentence2_lang,sentence1,sentence2,model,direction,dataset,annotator_lang
0,7353368635255019224,563,eng,eng,"Their predecessors, the Australopithecus did n...","Their ancestor, Australopithecus, was not equa...",Cross_Lingual_Calibration,backward_calibration,Cross_Lingual_Calibration,zul
1,5317902272397337966,325,zul,eng,"Emananini adlulele wokuvota, amaphesentu angu-...","At the extremes of the poll, 29 per cent of th...",human,backward,devtest,zul
2,7241908964355042255,151,zul,eng,Umkhulumeli we-Medecines Sans Frontiere u-Rich...,"Richard Veerman , a spokesman for Medecines Sa...",m2m100,backward,devtest,zul
3,494573535392983270,403,zul,eng,Okubhalwe ngemuva eNcazelweni Yokuzimela bekun...,"Written behind the Apostolic Exhortation was ""...",m2m100,backward,devtest,zul
4,15759707055590300438,645,eng,zul,"People now write messages on computer screens,...",Abantu manje babhala imilayezo ezikrinini zama...,human,forward,devtest,zul
...,...,...,...,...,...,...,...,...,...,...
4449,15948662869656859630,171,eng,amh,"In late 2015, TogiNet established AstroNet Rad...",በ 2015 መገባደጃ ላይ ቶጊኔት አስትሮኔት ሬዲዮን እንደ አንድ ንዑስ ጣ...,human,forward,devtest,amh
4450,1344353142545256770,611,amh,eng,በአደጉ ሀገራት አንዳንድ ጊዜ ስለ ውሃ ጥራት እና ስለ ድልድዮች መውደቅ ...,In developed countries you seldom hear similar...,human,backward,devtest,amh
4451,12649507862239568202,805,eng,amh,Visibility may also be restricted by falling o...,ዕይታ በሚወድቅ ወይም በሚነፍስ በረዶ ወይም በውሃ መጤዝ ወይም በመኪና መ...,human,forward,devtest,amh
4452,16384303385687987122,647,eng,amh,The fission bomb works on the principle that i...,የፍንዳታ ቦንብ የሚሠራው ሁሉንም ኃይል በሚቻለው ፕሮቶን እና ኒውትሮን ወ...,human,forward,devtest,amh


## Import score data

In [8]:
!pwd

/private/home/dlicht/fairseq-py/examples/nllb/evaluation/XSTS_eval/2021_09_15_next_14/analysis_notebooks


In [9]:
# !mkdir /data/users/dlicht/scratch/set_1_final/
# !mkdir /data/users/dlicht/scratch/set_1_raw/

In [10]:
# ! unzip "/data/users/dlicht/scratch/Final - with 1-point difference rule-20210812T231220Z-001.zip" -d /data/users/dlicht/scratch/
# ! unzip "/data/users/dlicht/scratch/Raw - without 1-point difference rule-20210813T172427Z-001.zip" -d /data/users/dlicht/scratch/

In [11]:
ls /private/home/dlicht/fairseq-py/examples/nllb/evaluation/XSTS_eval/2021_09_15_next_14/human_eval_sample_generation/master_copy -lah

total 24M
drwxrwxr-x 2 dlicht dlicht    0 Oct 19 15:43 ./
drwxrwxr-x 5 dlicht dlicht    0 Oct 19 15:43 ../
-rw-rw-r-- 1 dlicht dlicht 1.8M Oct 19 15:43 20210913_eval_NLLB_m2m100_amh.tsv
-rw-rw-r-- 1 dlicht dlicht 1.7M Oct 19 15:43 20210913_eval_NLLB_m2m100_ara.tsv
-rw-rw-r-- 1 dlicht dlicht 1.6M Oct 19 15:43 20210913_eval_NLLB_m2m100_azj.tsv
-rw-rw-r-- 1 dlicht dlicht 1.5M Oct 19 15:43 20210913_eval_NLLB_m2m100_bos.tsv
-rw-rw-r-- 1 dlicht dlicht 1.9M Oct 19 15:43 20210913_eval_NLLB_m2m100_bul.tsv
-rw-rw-r-- 1 dlicht dlicht 2.1M Oct 19 15:43 20210913_eval_NLLB_m2m100_hin.tsv
-rw-rw-r-- 1 dlicht dlicht 2.3M Oct 19 15:43 20210913_eval_NLLB_m2m100_kat.tsv
-rw-rw-r-- 1 dlicht dlicht 1.5M Oct 19 15:43 20210913_eval_NLLB_m2m100_por.tsv
-rw-rw-r-- 1 dlicht dlicht 1.5M Oct 19 15:43 20210913_eval_NLLB_m2m100_ron.tsv
-rw-rw-r-- 1 dlicht dlicht 1.5M Oct 19 15:43 20210913_eval_NLLB_m2m100_slv.tsv
-rw-rw-r-- 1 dlicht dlicht 1.7M Oct 19 15:43 20210913_eval_NLLB_m2m100_snd.tsv
-rw-rw-r-- 1 dlicht dlic

In [12]:
ls ../annotated_data -lah

total 35M
drwxrwxr-x 4 dlicht dlicht   0 Oct 19 15:43  ./
drwxrwxr-x 5 dlicht dlicht   0 Oct 19 15:43  ../
-rw-rw-r-- 1 dlicht dlicht 11M Oct 19 15:43  Delivery-20211015T230853Z-001.zip
-rw-rw-r-- 1 dlicht dlicht 13M Oct 19 15:43 'Final - with 1-point difference rule-20211015T230838Z-001.zip'
drwxrwxr-x 2 dlicht dlicht   0 Oct 19 15:43  LIONBRIDGE_Delivery_Raw/
drwxrwxr-x 2 dlicht dlicht   0 Oct 19 15:43  MORAVIA_Delivery_Adj/
-rw-rw-r-- 1 dlicht dlicht 12M Oct 19 15:43 'Raw - without 1-point difference rule-20211015T233125Z-001.zip'
-rw-rw-r-- 1 dlicht dlicht 588 Oct 19 15:43  readme.txt


In [13]:
# IMPORT MORAVIA

DIR = "/private/home/dlicht/fairseq-py/examples/nllb/evaluation/XSTS_eval/2021_09_15_next_14/annotated_data/MORAVIA_Delivery_Adj"
df_file_import = pd.DataFrame()

for f in os.listdir(f"{DIR}"):
    print(f)
    try:
        temp_df = pd.read_csv(DIR+"/"+f, sep='\t', error_bad_lines=False)
        # temp_df = pd.read_csv(DIR+"/"+f, sep='\t', on_bad_lines='error', verbose=True)
        print(temp_df.shape)
        temp_df['annotator_lang'] = f[-9:-6]
        temp_df['vendor'] = "MORAVIA"
        df_file_import = df_file_import.append(temp_df)
    except:
        print(f"file import ERROR {f}")

df_file_import.tail()

20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_amh_1.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_amh_2.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_amh_3.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_ara_1.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_ara_2.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_ara_3.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_bul_1.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_bul_2.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_bul_3.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_kat_1.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_kat_2.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_kat_3.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_ron_1.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_ron_2.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_ron_3.tsv
(4454, 9)
20210915_MORAVIA_ADHOC_NLLB-HE_H2-R1_2021_slv_1.tsv
(4454, 9)
20210915

,fbid,sentence1_lang,sentence2_lang,sentence1,sentence1_incoherence,sentence2,sentence2_incoherence,sts_score,evaluator_id,annotator_lang,vendor
4449,11007206146729409094,snd,eng,پاڻي کي صاف ڪرڻ لاءِ مختلف طريقا آهن، مخصوص خط...,0,"There are different ways of purifying water, s...",0,5,ZS,snd,MORAVIA
4450,13117101197243282333,eng,snd,Travel agents often have deals with specific h...,0,مسافر عام طور تي مخصوص هتلن سان ڪم ڪري رهيا آه...,0,4,ZS,snd,MORAVIA
4451,16398295403275621272,snd,eng,پولينڊ جا گهٽ نظر وارو اسڪائير ماچائي ڪريزيل ۽...,0,They had brought the knowledge of theliving Go...,0,1,ZS,snd,MORAVIA
4452,17413315577666148688,eng,eng,"Generally speaking, two behaviors can emerge a...",0,Jhfxyjfhjhjhkhhdtdtfnhjhjb hvydhbn hey nah gun...,1,1,ZS,snd,MORAVIA
4453,892557528745161778,eng,snd,"Popular sports include football, basketball, v...",0,مشهور ورزشن ۾ فٽبال ، بسڪٽبال ، ووليبال ، پاڻي...,0,5,ZS,snd,MORAVIA


In [14]:
df_file_import.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93534 entries, 0 to 4453
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   fbid                   93534 non-null  uint64
 1   sentence1_lang         93534 non-null  object
 2   sentence2_lang         93534 non-null  object
 3   sentence1              93534 non-null  object
 4   sentence1_incoherence  93534 non-null  int64 
 5   sentence2              93534 non-null  object
 6   sentence2_incoherence  93534 non-null  int64 
 7   sts_score              93534 non-null  int64 
 8   evaluator_id           93534 non-null  object
 9   annotator_lang         93534 non-null  object
 10  vendor                 93534 non-null  object
dtypes: int64(3), object(7), uint64(1)
memory usage: 8.6+ MB


had to install support for xlsx:

pip3 install openpyxl and add , "engine='openpyxl'"

In [15]:
# !pip3 install openpyxl 

In [16]:
# IMPORT LIONBRIDGE
## into the SAME dataframe (column names will need to agree)

DIR = "/private/home/dlicht/fairseq-py/examples/nllb/evaluation/XSTS_eval/2021_09_15_next_14/annotated_data/LIONBRIDGE_Delivery_Raw"
# df_file_import = pd.DataFrame()

for f in os.listdir(f"{DIR}"):
    print(f)
    try:
        temp_df = pd.read_excel(DIR+"/"+f, engine='openpyxl')
        # temp_df = pd.read_csv(DIR+"/"+f, sep='\t', error_bad_lines=False)
        # temp_df = pd.read_csv(DIR+"/"+f, sep='\t', on_bad_lines='error', verbose=True)
        print(temp_df.shape)
        temp_df['annotator_lang'] = f[-10:-7]
        temp_df['vendor'] = "LIONBRIDGE"
        df_file_import = df_file_import.append(temp_df)
    except:
        print(f"file import ERROR {f}")

df_file_import.tail()

20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_azj_1.xlsx
(4454, 9)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_azj_2.xlsx
(4454, 8)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_azj_3.xlsx
(4454, 9)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_bos_1.xlsx
(4454, 9)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_bos_2.xlsx
(4454, 9)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_bos_3.xlsx
(4454, 9)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_hin_1.xlsx
(4454, 9)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_hin_2.xlsx
(4454, 9)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_hin_3.xlsx
(4454, 9)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_por_1.xlsx
(4454, 9)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_por_2.xlsx
(4454, 9)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_por_3.xlsx
(4454, 9)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_swh_1.xlsx
(4454, 9)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_swh_2.xlsx
(4454, 8)
20211015_LIONBRIDGE_ADHOC_NLLB-HE_H2-R1_2021_swh_3.xlsx
(4454, 9)
20211015_L

,fbid,sentence1_lang,sentence2_lang,sentence1,sentence1_incoherence,sentence2,sentence2_incoherence,sts_score,evaluator_id,annotator_lang,vendor
4449,1192476332129914486,zul,eng,Ukubhikisha okuba nodlame ngezinye izikhathi k...,0,Loyalty is sometimes deleted by the absence of...,0,5,256024,zul,LIONBRIDGE
4450,3942416496547317383,eng,eng,Anyone who's going to drive at high latitudes ...,0,It is likely that anyone driving at high altit...,0,5,256024,zul,LIONBRIDGE
4451,16726181206568233591,zul,eng,"Ngakolunye uhlangothi, umuntu uyaguquka abe ng...",0,"On the other hand , a person will turn into an...",0,5,256024,zul,LIONBRIDGE
4452,10374372936450275266,eng,eng,Moldova is a multi-ethnic republic that has su...,0,The Maldivian Republic is a tribe and a number...,0,5,256024,zul,LIONBRIDGE
4453,3819223443325538966,eng,zul,"However, these plans were rendered obsolete ne...",0,"Nokho, lezi zinhlelo zaphelelwa yisikhathi cis...",0,5,256024,zul,LIONBRIDGE


In [17]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4454 entries, 0 to 4453
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   fbid                   4454 non-null   uint64
 1   sentence1_lang         4454 non-null   object
 2   sentence2_lang         4454 non-null   object
 3   sentence1              4454 non-null   object
 4   sentence1_incoherence  4454 non-null   int64 
 5   sentence2              4454 non-null   object
 6   sentence2_incoherence  4454 non-null   int64 
 7   sts_score              4454 non-null   int64 
 8   evaluator_id           4454 non-null   int64 
 9   annotator_lang         4454 non-null   object
 10  vendor                 4454 non-null   object
dtypes: int64(4), object(6), uint64(1)
memory usage: 382.9+ KB


In [18]:
# pd.read_excel(DIR+"/"+f, engine='openpyxl')

In [19]:
df_file_import.groupby(['annotator_lang']).count()/(3*2*(1000+500+707))

,fbid,sentence1_lang,sentence2_lang,sentence1,sentence1_incoherence,sentence2,sentence2_incoherence,sts_score,evaluator_id,vendor
annotator_lang,,,,,,,,,,
amh,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062
ara,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062
azj,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,0.672708,1.009062
bos,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062
bul,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062
hin,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062
kat,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062
por,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062
ron,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062,1.009062


In [20]:
df_file_import.groupby(['annotator_lang']).count()/(3*1*(4454))

,fbid,sentence1_lang,sentence2_lang,sentence1,sentence1_incoherence,sentence2,sentence2_incoherence,sts_score,evaluator_id,vendor
annotator_lang,,,,,,,,,,
amh,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
ara,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
azj,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.666667,1.0
bos,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
bul,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
hin,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
kat,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
por,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
ron,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0


In [21]:
df_file_import.groupby(['sentence1_lang','sentence2_lang','vendor']).count()/(3*1*(4455))

fbid  sentence1  \
sentence1_lang sentence2_lang vendor                            
amh            eng            MORAVIA     0.387654   0.387654   
ara            eng            MORAVIA     0.387654   0.387654   
azj            eng            LIONBRIDGE  0.387654   0.387654   
bos            eng            LIONBRIDGE  0.387654   0.387654   
bul            eng            MORAVIA     0.387654   0.387654   
eng            amh            MORAVIA     0.387654   0.387654   
               ara            MORAVIA     0.387654   0.387654   
               azj            LIONBRIDGE  0.387654   0.387654   
               bos            LIONBRIDGE  0.387654   0.387654   
               bul            MORAVIA     0.387654   0.387654   
               eng            LIONBRIDGE  1.571268   1.571268   
                              MORAVIA     1.571268   1.571268   
               hin            LIONBRIDGE  0.387654   0.387654   
               kat            MORAVIA     0.387654   0.387654   
               por            LIONBRIDGE  0.387654   0.387654   
               ron            MORAVIA     0.387654   0.387654   
               slv            MORAVIA     0.387654   0.387654   
               snd            MORAVIA     0.387654   0.387654   
               swh            LIONBRIDGE  0.387654   0.387654   
               urd            LIONBRIDGE  0.387654   0.387654   
               zul            LIONBRIDGE  0.387654   0.387654   
hin            eng            LIONBRIDGE  0.387654   0.387654   
kat            eng            MORAVIA     0.387654   0.387654   
por            eng            LIONBRIDGE  0.387654   0.387654   
ron            eng            MORAVIA     0.387654   0.387654   
slv            eng            MORAVIA     0.387654   0.387654   
snd            eng            MORAVIA     0.387654   0.387654   
swh            eng            LIONBRIDGE  0.387654   0.387654   
urd            eng            LIONBRIDGE  0.387654   0.387654   
zul            eng            LIONBRIDGE  0.387654   0.387654   

                                          sentence1_incoherence  sentence2  \
sentence1_lang sentence2_lang vendor                                         
amh            eng            MORAVIA                  0.387654   0.387654   
ara            eng            MORAVIA                  0.387654   0.387654   
azj            eng            LIONBRIDGE               0.387654   0.387654   
bos            eng            LIONBRIDGE               0.387654   0.387654   
bul            eng            MORAVIA                  0.387654   0.387654   
eng            amh            MORAVIA                  0.387654   0.387654   
               ara            MORAVIA                  0.387654   0.387654   
               azj            LIONBRIDGE               0.387654   0.387654   
               bos            LIONBRIDGE               0.387654   0.387654   
               bul            MORAVIA                  0.387654   0.387654   
               eng            LIONBRIDGE               1.571268   1.571268   
                              MORAVIA                  1.571268   1.571268   
               hin            LIONBRIDGE               0.387654   0.387654   
               kat            MORAVIA                  0.387654   0.387654   
               por            LIONBRIDGE               0.387654   0.387654   
               ron            MORAVIA                  0.387654   0.387654   
               slv            MORAVIA                  0.387654   0.387654   
               snd            MORAVIA                  0.387654   0.387654   
               swh            LIONBRIDGE               0.387654   0.387654   
               urd            LIONBRIDGE               0.387654   0.387654   
               zul            LIONBRIDGE               0.387654   0.387654   
hin            eng            LIONBRIDGE               0.387654   0.387654   
kat            eng            MORAVIA                  0.387654   0.387654   
por            

In [22]:
df_file_import.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187068 entries, 0 to 4453
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   fbid                   187068 non-null  uint64
 1   sentence1_lang         187068 non-null  object
 2   sentence2_lang         187068 non-null  object
 3   sentence1              187068 non-null  object
 4   sentence1_incoherence  187068 non-null  int64 
 5   sentence2              187068 non-null  object
 6   sentence2_incoherence  187068 non-null  int64 
 7   sts_score              187068 non-null  int64 
 8   evaluator_id           178160 non-null  object
 9   annotator_lang         187068 non-null  object
 10  vendor                 187068 non-null  object
dtypes: int64(3), object(7), uint64(1)
memory usage: 17.1+ MB


In [23]:
# df_file_import = df_file_import.merge(df_source, on=['fbid','sentence1_lang','sentence2_lang'], how='left')
# df_file_import.tail()

In [75]:
# fix a strange id error (disk-bit error?) in one dataset
# 823032514845158000
# 823032514845158016
# FLORES 941 in Hindi

df_file_import['fbid'] = df_file_import['fbid'].apply(lambda x: 823032514845158000 if x==823032514845158016 else x)

In [76]:
df_mapped = df_source.merge(df_file_import, on=['fbid','sentence1_lang','sentence2_lang','annotator_lang'], how='outer', suffixes=('','_A'))
df_mapped.tail()

,fbid,FLORES_ID,sentence1_lang,sentence2_lang,sentence1,sentence2,model,direction,dataset,annotator_lang,sentence1_A,sentence1_incoherence,sentence2_A,sentence2_incoherence,sts_score,evaluator_id,vendor
187063,16384303385687987122,647,eng,amh,The fission bomb works on the principle that i...,የፍንዳታ ቦንብ የሚሠራው ሁሉንም ኃይል በሚቻለው ፕሮቶን እና ኒውትሮን ወ...,human,forward,devtest,amh,The fission bomb works on the principle that i...,0,የፍንዳታ ቦንብ የሚሠራው ሁሉንም ኃይል በሚቻለው ፕሮቶን እና ኒውትሮን ወ...,0,5,Bab-RG.Y,MORAVIA
187064,16384303385687987122,647,eng,amh,The fission bomb works on the principle that i...,የፍንዳታ ቦንብ የሚሠራው ሁሉንም ኃይል በሚቻለው ፕሮቶን እና ኒውትሮን ወ...,human,forward,devtest,amh,The fission bomb works on the principle that i...,0,የፍንዳታ ቦንብ የሚሠራው ሁሉንም ኃይል በሚቻለው ፕሮቶን እና ኒውትሮን ወ...,0,5,Bab-A_TM,MORAVIA
187065,10869730886167153991,180,eng,amh,The final match of the series will take place ...,የስፕሪንግቦክስ አውስትራሊያ በሚገጥምበት ጊዜ፣ የተከታታይ የመጨረሻ ግጥሚ...,human,forward,devtest,amh,The final match of the series will take place ...,0,የስፕሪንግቦክስ አውስትራሊያ በሚገጥምበት ጊዜ፣ የተከታታይ የመጨረሻ ግጥሚ...,0,5,Bab-TS,MORAVIA
187066,10869730886167153991,180,eng,amh,The final match of the series will take place ...,የስፕሪንግቦክስ አውስትራሊያ በሚገጥምበት ጊዜ፣ የተከታታይ የመጨረሻ ግጥሚ...,human,forward,devtest,amh,The final match of the series will take place ...,0,የስፕሪንግቦክስ አውስትራሊያ በሚገጥምበት ጊዜ፣ የተከታታይ የመጨረሻ ግጥሚ...,0,5,Bab-RG.Y,MORAVIA
187067,10869730886167153991,180,eng,amh,The final match of the series will take place ...,የስፕሪንግቦክስ አውስትራሊያ በሚገጥምበት ጊዜ፣ የተከታታይ የመጨረሻ ግጥሚ...,human,forward,devtest,amh,The final match of the series will take place ...,0,የስፕሪንግቦክስ አውስትራሊያ በሚገጥምበት ጊዜ፣ የተከታታይ የመጨረሻ ግጥሚ...,0,5,Bab-A_TM,MORAVIA


In [77]:
df_mapped[df_mapped['sentence1_A'].isnull()]

,fbid,FLORES_ID,sentence1_lang,sentence2_lang,sentence1,sentence2,model,direction,dataset,annotator_lang,sentence1_A,sentence1_incoherence,sentence2_A,sentence2_incoherence,sts_score,evaluator_id,vendor


In [78]:
df_mapped[df_mapped['sentence1'].isnull()]

,fbid,FLORES_ID,sentence1_lang,sentence2_lang,sentence1,sentence2,model,direction,dataset,annotator_lang,sentence1_A,sentence1_incoherence,sentence2_A,sentence2_incoherence,sts_score,evaluator_id,vendor


In [81]:
df_mapped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187068 entries, 0 to 187067
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   fbid                   187068 non-null  uint64
 1   FLORES_ID              187068 non-null  int64 
 2   sentence1_lang         187068 non-null  object
 3   sentence2_lang         187068 non-null  object
 4   sentence1              187068 non-null  object
 5   sentence2              187068 non-null  object
 6   model                  187068 non-null  object
 7   direction              187068 non-null  object
 8   dataset                187068 non-null  object
 9   annotator_lang         187068 non-null  object
 10  sentence1_A            187068 non-null  object
 11  sentence1_incoherence  187068 non-null  int64 
 12  sentence2_A            187068 non-null  object
 13  sentence2_incoherence  187068 non-null  int64 
 14  sts_score              187068 non-null  int64 
 15  

In [82]:
df_mapped.count()

fbid                     187068
FLORES_ID                187068
sentence1_lang           187068
sentence2_lang           187068
sentence1                187068
sentence2                187068
model                    187068
direction                187068
dataset                  187068
annotator_lang           187068
sentence1_A              187068
sentence1_incoherence    187068
sentence2_A              187068
sentence2_incoherence    187068
sts_score                187068
evaluator_id             178160
vendor                   187068
dtype: int64

In [83]:
df_mapped.groupby(['vendor','sentence1_lang','sentence2_lang','annotator_lang']).count()

fbid  FLORES_ID  \
vendor     sentence1_lang sentence2_lang annotator_lang                    
LIONBRIDGE azj            eng            azj             5181       5181   
           bos            eng            bos             5181       5181   
           eng            azj            azj             5181       5181   
                          bos            bos             5181       5181   
                          eng            azj             3000       3000   
                                         bos             3000       3000   
                                         hin             3000       3000   
                                         por             3000       3000   
                                         swh             3000       3000   
                                         urd             3000       3000   
                                         zul             3000       3000   
                          hin            hin             5181       5181   
                          por            por             5181       5181   
                          swh            swh             5181       5181   
                          urd            urd             5181       5181   
                          zul            zul             5181       5181   
           hin            eng            hin             5181       5181   
           por            eng            por             5181       5181   
           swh            eng            swh             5181       5181   
           urd            eng            urd             5181       5181   
           zul            eng            zul             5181       5181   
MORAVIA    amh            eng            amh             5181       5181   
           ara            eng            ara             5181       5181   
           bul            eng            bul             5181       5181   
           eng            amh            amh             5181       5181   
                          ara            ara             5181       5181   
                          bul            bul             5181       5181   
                          eng            amh             3000       3000   
                                         ara             3000       3000   
                                         bul             3000       3000   
                                         kat             3000       3000   
                                         ron             3000       3000   
                                         slv             3000       3000   
                                         snd             3000       3000   
                          kat            kat             5181       5181   
                          ron            ron             5181       5181   
                          slv            slv             5181       5181   
                          snd            snd             5181       5181   
           kat            eng            kat             5181       5181   
           ron            eng            ron             5181       5181   
           slv            eng            slv             5181       5181   
           snd            eng            snd             5181       5181   

                                                         sentence1  sentence2  \
vendor     sentence1_lang sentence2_lang annotator_lang                         
LIONBRIDGE azj            eng            azj                  5181       5181   
           bos            eng            bos                  5181       5181   
           eng            azj            azj                  5181       5181   
                          bos            bos                  5181       5181   
                          eng            azj                  3000       3000   
                                         bos                  3000       3000   
                                         hin                  30

In [84]:
df_mapped.groupby(['vendor','sentence1_lang','sentence2_lang',]).count().sort_values('fbid')

fbid  FLORES_ID  sentence1  \
vendor     sentence1_lang sentence2_lang                                
LIONBRIDGE azj            eng              5181       5181       5181   
MORAVIA    ron            eng              5181       5181       5181   
           kat            eng              5181       5181       5181   
           eng            snd              5181       5181       5181   
                          slv              5181       5181       5181   
                          ron              5181       5181       5181   
                          kat              5181       5181       5181   
                          bul              5181       5181       5181   
                          ara              5181       5181       5181   
                          amh              5181       5181       5181   
           bul            eng              5181       5181       5181   
           ara            eng              5181       5181       5181   
           amh            eng              5181       5181       5181   
           slv            eng              5181       5181       5181   
LIONBRIDGE zul            eng              5181       5181       5181   
           swh            eng              5181       5181       5181   
           por            eng              5181       5181       5181   
           hin            eng              5181       5181       5181   
           eng            zul              5181       5181       5181   
                          urd              5181       5181       5181   
                          swh              5181       5181       5181   
                          por              5181       5181       5181   
                          hin              5181       5181       5181   
                          bos              5181       5181       5181   
                          azj              5181       5181       5181   
           bos            eng              5181       5181       5181   
           urd            eng              5181       5181       5181   
MORAVIA    snd            eng              5181       5181       5181   
           eng            eng             21000      21000      21000   
LIONBRIDGE eng            eng             21000      21000      21000   

                                          sentence2  model  direction  \
vendor     sentence1_lang sentence2_lang                                
LIONBRIDGE azj            eng                  5181   5181       5181   
MORAVIA    ron            eng                  5181   5181       5181   
           kat            eng                  5181   5181       5181   
           eng            snd                  5181   5181       5181   
                          slv                  5181   5181       5181   
                          ron                  5181   5181       5181   
                          kat                  5181   5181       5181   
                          bul                  5181   5181       5181   
                          ara                  5181   5181       5181   
                          amh                  5181   5181       5181   
           bul            eng                  5181   5181       5181   
           ara            eng                  5181   5181       5181   
           amh            eng                  5181   5181       5181   
           slv            eng                  5181   5181       5181   
LIONBRIDGE zul            eng                  5181   5181       5181   
           swh            eng                  5181   5181       5181   
           por            eng                  5181   5181       5181   
           hin            eng                  5181   5181       5181   
           eng            zul                  5181   5181       5181   
                          urd                  5181   5181       5181   
                          swh                  5181   5181       5181   
                          p

In [86]:
# df_file_import['flores_id'] = 
# df_file_import['model'] = df_file_import['id'].str.split('_', expand=True)[0]
# df_file_import['test_set']  = df_file_import['id'].str.split('_', expand=True)[1]
# df_file_import['flores_id'] = df_file_import['id'].str.split('_', expand=True)[2]

# df_file_import['sentence1'] = df_file_import['sentence1'].apply(lambda x: [x])
# df_file_import['sentence2'] = df_file_import['sentence2'].apply(lambda x: [x])

df_mapped['FLORES_ID'] = df_mapped['FLORES_ID'].fillna(-1)
df_mapped['FLORES_ID'] = df_mapped['FLORES_ID'].astype('int')
# df_mapped['model'] = df_mapped['model'].str.replace('backward','').str.replace('forward','')
df_mapped.sample()

,fbid,FLORES_ID,sentence1_lang,sentence2_lang,sentence1,sentence2,model,direction,dataset,annotator_lang,sentence1_A,sentence1_incoherence,sentence2_A,sentence2_incoherence,sts_score,evaluator_id,vendor
94719,9182620935479106183,603,eng,kat,Only mutations in germ-line cells can be passe...,მხოლოდ germ-line cells can be passed on to chi...,m2m100,forward,devtest,kat,Only mutations in germ-line cells can be passe...,0,მხოლოდ germ-line cells can be passed on to chi...,1,1,N.G.,MORAVIA


In [87]:
# these are items which failed to map back to the original dataset
df_mapped[df_mapped['FLORES_ID'] == -1]

,fbid,FLORES_ID,sentence1_lang,sentence2_lang,sentence1,sentence2,model,direction,dataset,annotator_lang,sentence1_A,sentence1_incoherence,sentence2_A,sentence2_incoherence,sts_score,evaluator_id,vendor


In [88]:
df_mapped.reset_index().groupby(['sentence1_lang','sentence2_lang','model']).count()

index   fbid  \
sentence1_lang sentence2_lang model                                     
amh            eng            human                       2145   2145   
                              m2m100                      3036   3036   
ara            eng            human                       2145   2145   
                              m2m100                      3036   3036   
azj            eng            human                       2145   2145   
                              m2m100                      3036   3036   
bos            eng            human                       2145   2145   
                              m2m100                      3036   3036   
bul            eng            human                       2145   2145   
                              m2m100                      3036   3036   
eng            amh            human                       2145   2145   
                              m2m100                      3036   3036   
               ara            human                       2145   2145   
                              m2m100                      3036   3036   
               azj            human                       2145   2145   
                              m2m100                      3036   3036   
               bos            human                       2145   2145   
                              m2m100                      3036   3036   
               bul            human                       2145   2145   
                              m2m100                      3036   3036   
               eng            Cross_Lingual_Calibration  42000  42000   
               hin            human                       2145   2145   
                              m2m100                      3036   3036   
               kat            human                       2145   2145   
                              m2m100                      3036   3036   
               por            human                       2145   2145   
                              m2m100                      3036   3036   
               ron            human                       2145   2145   
                              m2m100                      3036   3036   
               slv            human                       2145   2145   
                              m2m100                      3036   3036   
               snd            human                       2145   2145   
                              m2m100                      3036   3036   
               swh            human                       2145   2145   
                              m2m100                      3036   3036   
               urd            human                       2145   2145   
                              m2m100                      3036   3036   
               zul            human                       2145   2145   
                              m2m100                      3036   3036   
hin            eng            human                       2145   2145   
                              m2m100                      3036   3036   
kat            eng            human                       2145   2145   
                              m2m100                      3036   3036   
por            eng            human                       2145   2145   
                              m2m100                      3036   3036   
ron            eng            human                       2145   2145   
                              m2m100                      3036   3036   
slv            eng            human                       2145   2145   
                              m2m100                      3036   3036   
snd            eng            human                       2145   2145   
                              m2m100                      3036   3036   
swh            eng            human                       2145   2145   
                              m2m100                      3036   3036   
urd            eng            human       

In [89]:
df_mapped.reset_index().groupby(['model']).count()['FLORES_ID']

model
Cross_Lingual_Calibration    42000
human                        60060
m2m100                       85008
Name: FLORES_ID, dtype: int64

In [90]:
df_mapped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187068 entries, 0 to 187067
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   fbid                   187068 non-null  uint64
 1   FLORES_ID              187068 non-null  int64 
 2   sentence1_lang         187068 non-null  object
 3   sentence2_lang         187068 non-null  object
 4   sentence1              187068 non-null  object
 5   sentence2              187068 non-null  object
 6   model                  187068 non-null  object
 7   direction              187068 non-null  object
 8   dataset                187068 non-null  object
 9   annotator_lang         187068 non-null  object
 10  sentence1_A            187068 non-null  object
 11  sentence1_incoherence  187068 non-null  int64 
 12  sentence2_A            187068 non-null  object
 13  sentence2_incoherence  187068 non-null  int64 
 14  sts_score              187068 non-null  int64 
 15  

In [91]:
# df_mapped.drop(columns=['fbid'], inplace=True)
df_mapped.sample()

,fbid,FLORES_ID,sentence1_lang,sentence2_lang,sentence1,sentence2,model,direction,dataset,annotator_lang,sentence1_A,sentence1_incoherence,sentence2_A,sentence2_incoherence,sts_score,evaluator_id,vendor
79276,16027928815069446205,232,ron,eng,"Cântărețul Sanju Sharma a început spectacolul,...","Singer Sanju Sharma started the evening, follo...",human,backward,devtest,ron,"Cântărețul Sanju Sharma a început spectacolul,...",0,"Singer Sanju Sharma started the evening, follo...",0,4,C.D.,MORAVIA


# Bento Upload 

In [98]:
if write_file == True:
    df_mapped.to_csv(f"{output_folder}{output_filename}", sep="\t")

In [99]:
# if write_file == True:
#     bb.upload_data_to_hive(
#         'aml',
#         df_file_import,
#         tablename,
#         retention=30,
#         # ds_partition='2021-04-19',
#         oncall='latte_analytics',
#     )